# [선형회귀모델 실습 1: 모델링 및 해석]

##### jupyter notebook 단축키

- ctrl+enter: 셀 실행   
- shift+enter: 셀 실행 및 다음 셀 이동   
- alt+enter: 셀 실행, 다음 셀 이동, 새로운 셀 생성
- a: 상단에 새로운 셀 만들기
- b: 하단에 새로운 셀 만들기
- dd: 셀 삭제(x: 셀 삭제)


## 1. 모듈 불러오기

import 불러올 패키지명 as 그 패키지를 파이썬에서 사용할 이름

In [ ]:
from IPython.display import display, HTML

# 데이터 전처리 패키지
import numpy as np
import pandas as pd

# 기계학습 모델 구축 및 평가 패키지
import scipy as sp
import scipy.stats as stats

import statsmodels.api as sm
from statsmodels.formula.api import ols

from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# 데이터 시각화 패키지
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정
plt.rc('font', family='Malgun Gothic')

## 2. 데이터 불러오기: Boston Housing Data

#### 데이터 구조  
- 데이터: 1978년 보스턴 주택 가격  
- 관측치 개수: 506개
- 변수 개수: 설명변수 13개 / 반응변수 1개

#### 설명 변수(원인: 예측값을 설명할 수 있는 변수)      
- CRIM: 범죄율  
- INDUS: 비소매상업지역 면적 비율  
- NOX: 일산화질소 농도  
- RM: 주택당 방 수  
- LSTAT: 인구 중 하위 계층 비율  
- B: 인구 중 흑인 비율  
- PTRATIO: 학생/교사 비율  
- ZN: 25,000 평방피트를 초과 거주지역 비율  
- CHAS: 찰스강의 경계에 위치한 경우는 1, 아니면 0  
- AGE: 1940년 이전에 건축된 주택의 비율  
- RAD: 방사형 고속도로까지의 거리  
- DIS: 직업센터의 거리  
- TAX: 재산세율  

#### 반응 변수(결과: 예측하고자 하는 값)
- MEDV: 주택가격

In [ ]:
boston = load_boston()
print(boston.DESCR)

In [ ]:
boston.data.shape

In [ ]:
boston.data

In [ ]:
boston.feature_names

In [ ]:
boston.target

## 3. 데이터 전처리

$\mathbf{X}$: 설명변수/입력변수 <br>
$\mathbf{y}$: 반응변수/출력변수

- 데이터 프레임

In [ ]:
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = pd.DataFrame(boston.target, columns=['MEDV'])

In [ ]:
X

In [ ]:
X.head()

In [ ]:
y.head()

- 데이터 병합

In [ ]:
data = pd.concat([X, y], axis=1)

In [ ]:
data.head(10)

## 4. 탐색적 데이터 분석

### sns.pairplot을 이용하여 설명변수의 분포(히스토그램) 및 설명변수들 간의 관계(산점도)를 그리고, 특성을 파악

In [ ]:
sns.pairplot(data)
plt.show()

### 일부 변수만 선택하여 확인

In [ ]:
plt.figure(figsize=(10, 10))
sns.pairplot(data[['AGE', 'TAX', 'RM', 'MEDV']])
plt.show()

### 설명변수 간 상관계수(correlation coefficient) 확인

- data.corr(): 데이터 상관관계
- sns.heatmap(데이터, cmap= colormap 이름, annot=히트맵 안에 표시할 데이터)


In [ ]:
X.corr()

In [ ]:
plt.figure(figsize=(12, 12))
sns.heatmap(X.corr(), cmap=sns.color_palette("coolwarm", 10), annot=X.corr())
plt.show()

## 5. 모델링

### statsmodels의 OLS를 사용하여 선형회귀분석 시행 (OLS: Ordinary Least Squares)
 - OLS: 가장 기본적인 결정론적 선형 회귀 방법으로 잔차제곱합(RSS: Residual Sum of Squares)를 최소화하는 가중치(β1, β2 ...) 구하는 방법
 - 모델 선언: model = sm.OLS(Y데이터, X데이터)   
 - 모델 학습: model_trained = model.fit()

In [ ]:
model = sm.OLS(data['MEDV'], data.drop(['MEDV'], axis=1))

In [ ]:
model_trained = model.fit()

## 6. 모델 해석
- R-squared (결정계수,coefficient of determination):모형의 성능
- coef (회귀계수): X가 한단위 증가할 때 Y의 변화량
- P>[t] (p-value):0.1 이하일 때 변수가 유의미

In [ ]:
display(model_trained.summary())

### 분산분석 (ANOVA: Analysis of Variance) 시행

- 종속변수의 분산과 독립변수의 분산간의 관계를 사용하여 선형회귀분석의 성능을 평가   


In [ ]:
model_2 = ols('MEDV ~ CRIM + ZN + INDUS + CHAS + NOX + RM + AGE + DIS + RAD + TAX + PTRATIO + B + LSTAT - 1', data=data)
model_2_trained = model_2.fit()

In [ ]:
display(model_2_trained.summary())

In [ ]:
sm.stats.anova_lm(model_2_trained)

## 7. 실제값 vs. 모델 출력 값 비교


### 실제값 - 모델 출력 값 산점도를 통한 비교

In [ ]:
model_2_trained.fittedvalues

In [ ]:
y_predicted = model_2_trained.fittedvalues
y_predicted.head()

In [ ]:
plt.figure(figsize=(8, 8))
plt.title('실제값 vs. 모델 출력 값')
plt.scatter(y, y_predicted)
plt.xlabel('실제값', size=16)
plt.ylabel('모델 출력 값', size=16)
plt.xlim(-5, 55)
plt.ylim(-5, 55)
plt.show()

### 모델 출력 값 평가를 위한 지표들

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true-y_pred)/y_true))*100

### <center> Mean Squared Error (평균 제곱 오차): $\frac{1}{n} \sum_{i=1}^{n} (y_{i} - \hat{y}_{i})^{2}$   

In [ ]:
print(mean_squared_error(y, y_predicted))

### <center> Root Mean Squared Error (제곱근 평균 제곱 오차): $\sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_{i} - \hat{y}_{i})^{2}}$

In [ ]:
print(np.sqrt(mean_squared_error(y,y_predicted)))

### <center>Mean Absolute Error (평균 절대 오차): $\frac{1}{n} \sum_{i=1}^{n} |y_{i} - \hat{y}_{i}|$



In [ ]:
print(mean_absolute_error(y, y_predicted))

### <center> Mean Absolute Percentage Error (평균 절대 백분율 오차): $\frac{1}{n}\sum_{i=1}^{n} \left|\frac{y_{i} - \hat{y}_{i}}{y_{i}} \right| \times 100\%$

In [ ]:
print(mean_absolute_percentage_error(y, y_predicted))

### <center> Correlation Coefficient (상관계수)

In [ ]:
print(np.corrcoef(data['MEDV'].values.flatten(), y_predicted))